# MongoDB Joins

In MongoDB joins happen within the context of the aggregate pipeline.

Unlike relational databases where there is a variety of joins to utilize, MongoDB has only left outer equi-joins at it's disposal.

The most basic form of this syntax looks like so:

```json
db.COLLECTION_NAME.aggregate([{
    $lookup: {
        from: "COLLECTION_TO_JOIN",
        localField: "localField",
        foreignField: "foreignField",
        as: "jsonKey"
    }
}]);
```

## Example 1 -  A Simple Join Using Foreign Keys

1. Let's get our collections

In [1]:
from pymongo import MongoClient
from pprint import pprint

import pymongo

# Initialize a Mongo Client
#################################################
# Update UPDATE-ME in the connection code
#################################################
# Client 1 - mongodb-1.dsa.missouri.edu
# Client 2 - mongodb-2.dsa.missouri.edu
# Client 3 - mongodb-3.dsa.missouri.edu
# Client 4 - mongodb-4.dsa.missouri.edu
#################################################
#
client = MongoClient('mongodb-2.dsa.missouri.edu',
                     username='mlLargeReader',
                     password='mlLargeReader.create.role',
                     authSource='mlLarge')

db = client.mlLarge

# Get list of collections
cursor = db.collection_names()
for document in cursor:
    pprint(document)

'movies'
'links'
'tags'
'ratings'
'genome-scores'
'genome-tags'
'users'


2. Let's join two collections:

In [3]:
db.movies.find_one()

{'_id': ObjectId('5b18b242d698289b410b776c'),
 'genres': 'Comedy|Drama|Romance',
 'movieId': 4,
 'title': 'Waiting to Exhale (1995)'}

In [4]:
db.links.find_one()

{'_id': ObjectId('5b18b30bd698289b413d5e60'),
 'imdbId': 113497,
 'movieId': 2,
 'tmdbId': 8844}

In [2]:
documents = db.movies.aggregate([{
    # Let's bring in the links collection using movieId as our key
    "$lookup": {
        "from": "links",
        "localField": "movieId",
        "foreignField": "movieId",
        "as": "links"
    }
},{
    # Let's limit ourselves to something reasonable, just one document for now
    "$limit": 1
}]);

for document in documents:
    pprint(document)

{'_id': ObjectId('5b18b242d698289b410b776c'),
 'genres': 'Comedy|Drama|Romance',
 'links': [{'_id': ObjectId('5b18b30bd698289b413d5e62'),
            'imdbId': 114885,
            'movieId': 4,
            'tmdbId': 31357}],
 'movieId': 4,
 'title': 'Waiting to Exhale (1995)'}


**Note:** The joined tables are created as children of the root object, in Mongo v3.6+ you can use the $mergeObjects directive to merge them with the root object.

## Example 2 - Multiple Joins

In [5]:
db.ratings.find_one()

{'_id': ObjectId('5b18b242d698289b410be21d'),
 'movieId': 32,
 'rating': 3.5,
 'timestamp': 1112484819,
 'userId': 1}

In [6]:
documents = db.movies.aggregate([{
    # Let's join the ratings collection using movieId
    "$lookup": {
        "from": "ratings",
        "localField": "movieId",
        "foreignField": "movieId",
        "as": "ratings"
    }
}, {
    # Specify that we only want movies that have ratings
    "$match": {
        "ratings": {
            "$ne": []
        }
    }
}, {
    # Limit ourselves to one movie
    "$limit": 1
}, {
    # Bring in the links for our movie
    "$lookup": {
        "from": "links",
        "localField": "movieId",
        "foreignField": "movieId",
        "as": "links"
    }
}]);
#{
#     "$unwind": "$ratings"
# },{
#     "$limit": 3
# }]);

for document in documents:
    pprint(document)

{'_id': ObjectId('5b18b242d698289b410b776c'),
 'genres': 'Comedy|Drama|Romance',
 'links': [{'_id': ObjectId('5b18b30bd698289b413d5e62'),
            'imdbId': 114885,
            'movieId': 4,
            'tmdbId': 31357}],
 'movieId': 4,
 'ratings': [{'_id': ObjectId('5b18b242d698289b410bf34a'),
              'movieId': 4,
              'rating': 2.0,
              'timestamp': 951694198,
              'userId': 41},
             {'_id': ObjectId('5b18b242d698289b410bfddd'),
              'movieId': 4,
              'rating': 3.0,
              'timestamp': 1197068409,
              'userId': 61},
             {'_id': ObjectId('5b18b242d698289b410c2fb2'),
              'movieId': 4,
              'rating': 3.0,
              'timestamp': 1038801803,
              'userId': 156},
             {'_id': ObjectId('5b18b242d698289b410c38af'),
              'movieId': 4,
              'rating': 3.0,
              'timestamp': 842735262,
              'userId': 158},
             {'_id': Obj

              'rating': 4.0,
              'timestamp': 831200706,
              'userId': 7710},
             {'_id': ObjectId('5b18b24ed698289b411d3fb1'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 866869679,
              'userId': 7757},
             {'_id': ObjectId('5b18b24ed698289b411d5f2d'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 953672429,
              'userId': 7805},
             {'_id': ObjectId('5b18b24ed698289b411d612d'),
              'movieId': 4,
              'rating': 5.0,
              'timestamp': 848500225,
              'userId': 7807},
             {'_id': ObjectId('5b18b24ed698289b411d9ef0'),
              'movieId': 4,
              'rating': 5.0,
              'timestamp': 831553451,
              'userId': 7924},
             {'_id': ObjectId('5b18b24ed698289b411daf75'),
              'movieId': 4,
              'rating': 5.0,
              'timestamp': 1069100927,
       

              'timestamp': 832779379,
              'userId': 13259},
             {'_id': ObjectId('5b18b256d698289b412a7495'),
              'movieId': 4,
              'rating': 3.0,
              'timestamp': 880062477,
              'userId': 13576},
             {'_id': ObjectId('5b18b256d698289b412abb24'),
              'movieId': 4,
              'rating': 3.0,
              'timestamp': 988446090,
              'userId': 13680},
             {'_id': ObjectId('5b18b257d698289b412b2282'),
              'movieId': 4,
              'rating': 3.0,
              'timestamp': 841559273,
              'userId': 13864},
             {'_id': ObjectId('5b18b257d698289b412b2ee8'),
              'movieId': 4,
              'rating': 3.0,
              'timestamp': 882730289,
              'userId': 13891},
             {'_id': ObjectId('5b18b257d698289b412b47d7'),
              'movieId': 4,
              'rating': 3.0,
              'timestamp': 834193130,
              'userId': 13934},


              'rating': 3.0,
              'timestamp': 830256941,
              'userId': 20205},
             {'_id': ObjectId('5b18b260d698289b41396287'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 1101367052,
              'userId': 20217},
             {'_id': ObjectId('5b18b260d698289b41397c01'),
              'movieId': 4,
              'rating': 3.0,
              'timestamp': 855192868,
              'userId': 20257},
             {'_id': ObjectId('5b18b260d698289b4139944c'),
              'movieId': 4,
              'rating': 2.0,
              'timestamp': 958109067,
              'userId': 20285},
             {'_id': ObjectId('5b18b260d698289b4139b768'),
              'movieId': 4,
              'rating': 1.0,
              'timestamp': 843657410,
              'userId': 20336},
             {'_id': ObjectId('5b18b260d698289b4139bbb9'),
              'movieId': 4,
              'rating': 2.0,
              'timestamp': 965939947,
  

             {'_id': ObjectId('5b18b26ad698289b41485909'),
              'movieId': 4,
              'rating': 3.0,
              'timestamp': 835807807,
              'userId': 26985},
             {'_id': ObjectId('5b18b26ad698289b41487727'),
              'movieId': 4,
              'rating': 2.0,
              'timestamp': 962303598,
              'userId': 27040},
             {'_id': ObjectId('5b18b26ad698289b414886d7'),
              'movieId': 4,
              'rating': 1.0,
              'timestamp': 1356390435,
              'userId': 27053},
             {'_id': ObjectId('5b18b26ad698289b4148a12b'),
              'movieId': 4,
              'rating': 2.5,
              'timestamp': 1096133021,
              'userId': 27073},
             {'_id': ObjectId('5b18b26ad698289b4148ada4'),
              'movieId': 4,
              'rating': 5.0,
              'timestamp': 844852864,
              'userId': 27085},
             {'_id': ObjectId('5b18b26ad698289b4148d474'),
         

...

             {'_id': ObjectId('5b18b2e3d698289b4100f2b1'),
              'movieId': 4,
              'rating': 1.0,
              'timestamp': 830830515,
              'userId': 111021},
             {'_id': ObjectId('5b18b2e3d698289b4100fdb3'),
              'movieId': 4,
              'rating': 3.0,
              'timestamp': 969788926,
              'userId': 111051},
             {'_id': ObjectId('5b18b2e3d698289b41014535'),
              'movieId': 4,
              'rating': 3.0,
              'timestamp': 1019688890,
              'userId': 111156},
             {'_id': ObjectId('5b18b2e3d698289b41017bfe'),
              'movieId': 4,
              'rating': 5.0,
              'timestamp': 833464534,
              'userId': 111262},
             {'_id': ObjectId('5b18b2e3d698289b4101800c'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 950072341,
              'userId': 111272},
             {'_id': ObjectId('5b18b2e3d698289b410188ea'),
     

              'userId': 118236},
             {'_id': ObjectId('5b18b2edd698289b4111254f'),
              'movieId': 4,
              'rating': 3.0,
              'timestamp': 851636854,
              'userId': 118275},
             {'_id': ObjectId('5b18b2edd698289b41114fa6'),
              'movieId': 4,
              'rating': 3.0,
              'timestamp': 838489603,
              'userId': 118364},
             {'_id': ObjectId('5b18b2eed698289b411189b8'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 844073357,
              'userId': 118484},
             {'_id': ObjectId('5b18b2eed698289b4111b73f'),
              'movieId': 4,
              'rating': 1.0,
              'timestamp': 844955641,
              'userId': 118560},
             {'_id': ObjectId('5b18b2eed698289b4111ba9a'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 1080707341,
              'userId': 118568},
             {'_id': ObjectId('

              'timestamp': 845554349,
              'userId': 124644},
             {'_id': ObjectId('5b18b2f7d698289b411f3070'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 832252147,
              'userId': 124660},
             {'_id': ObjectId('5b18b2f7d698289b411f3aab'),
              'movieId': 4,
              'rating': 3.0,
              'timestamp': 848097017,
              'userId': 124682},
             {'_id': ObjectId('5b18b2f7d698289b411f47c9'),
              'movieId': 4,
              'rating': 3.5,
              'timestamp': 1177859916,
              'userId': 124711},
             {'_id': ObjectId('5b18b2f7d698289b411f8bfa'),
              'movieId': 4,
              'rating': 2.0,
              'timestamp': 967659170,
              'userId': 124831},
             {'_id': ObjectId('5b18b2f7d698289b411fa4f4'),
              'movieId': 4,
              'rating': 3.0,
              'timestamp': 974600605,
              'userId': 12

             {'_id': ObjectId('5b18b300d698289b412d236f'),
              'movieId': 4,
              'rating': 2.0,
              'timestamp': 844647119,
              'userId': 131063},
             {'_id': ObjectId('5b18b300d698289b412d32d6'),
              'movieId': 4,
              'rating': 2.0,
              'timestamp': 831647636,
              'userId': 131088},
             {'_id': ObjectId('5b18b300d698289b412d4eb5'),
              'movieId': 4,
              'rating': 3.0,
              'timestamp': 844762597,
              'userId': 131144},
             {'_id': ObjectId('5b18b300d698289b412d76b9'),
              'movieId': 4,
              'rating': 3.0,
              'timestamp': 835054617,
              'userId': 131215},
             {'_id': ObjectId('5b18b300d698289b412daf95'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 839409753,
              'userId': 131292},
             {'_id': ObjectId('5b18b300d698289b412e01c5'),
      

              'userId': 135855},
             {'_id': ObjectId('5b18b307d698289b4137b196'),
              'movieId': 4,
              'rating': 3.0,
              'timestamp': 1023064867,
              'userId': 135888},
             {'_id': ObjectId('5b18b307d698289b4137c3aa'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 1033674012,
              'userId': 135921},
             {'_id': ObjectId('5b18b307d698289b4137f14c'),
              'movieId': 4,
              'rating': 1.0,
              'timestamp': 845040607,
              'userId': 136003},
             {'_id': ObjectId('5b18b307d698289b4138163c'),
              'movieId': 4,
              'rating': 2.0,
              'timestamp': 901275476,
              'userId': 136057},
             {'_id': ObjectId('5b18b307d698289b413821b5'),
              'movieId': 4,
              'rating': 2.5,
              'timestamp': 1166632406,
              'userId': 136082},
             {'_id': ObjectId

## Example 3 - Joins with filtering

**Note:** In version 3.6+ of MongoDB you can pass an aggregate pipeline to the `$lookup` directive allowing you to filter those results directly, this is not possible in our version (3.4.15) of mongo.

In [7]:
documents = db.movies.aggregate([{
    # Let's join the ratings collection using movieId
    "$lookup": {
        "from": "ratings",
        "localField": "movieId",
        "foreignField": "movieId",
        "as": "ratings"
    }
}, {
    # Specify that we only want movies that have ratings
    "$match": {
        "ratings": {
            "$ne": []
        }
    }
}, {
    # Limit ourselves to one movie
    "$limit": 1
}, {
    # Bring in the links for our movie
    "$lookup": {
        "from": "links",
        "localField": "movieId",
        "foreignField": "movieId",
        "as": "links"
    }
}, {
    # Filter the ratings and specify our columns.
    # Note: There is no '*' specifier as in PostgresQL and most relational databases
    # you must explicitly list all desired columns if you project any of them.
    "$project": {
        "movieId": 1,
        "title": 1,
        "genres": 1,
        "links": 1,
        "ratings": {
            "$filter": {
                "input": "$ratings",
                "as": "rating",
                "cond": {
                    "$gte": ["$$rating.rating", 4.0]
                }
            },
        }
    }
}]);

for document in documents:
    pprint(document)

{'_id': ObjectId('5b18b242d698289b410b776c'),
 'genres': 'Comedy|Drama|Romance',
 'links': [{'_id': ObjectId('5b18b30bd698289b413d5e62'),
            'imdbId': 114885,
            'movieId': 4,
            'tmdbId': 31357}],
 'movieId': 4,
 'ratings': [{'_id': ObjectId('5b18b243d698289b410cc4e9'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 945303743,
              'userId': 417},
             {'_id': ObjectId('5b18b243d698289b410d0093'),
              'movieId': 4,
              'rating': 5.0,
              'timestamp': 839943473,
              'userId': 518},
             {'_id': ObjectId('5b18b243d698289b410d2d8f'),
              'movieId': 4,
              'rating': 5.0,
              'timestamp': 848681183,
              'userId': 601},
             {'_id': ObjectId('5b18b243d698289b410d3519'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 829414367,
              'userId': 611},
             {'_id': Obj

             {'_id': ObjectId('5b18b263d698289b413dfa6d'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 831670668,
              'userId': 22389},
             {'_id': ObjectId('5b18b264d698289b413f0337'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 939243562,
              'userId': 22898},
             {'_id': ObjectId('5b18b264d698289b413f2390'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 902218386,
              'userId': 22941},
             {'_id': ObjectId('5b18b264d698289b413f998d'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 834843826,
              'userId': 23108},
             {'_id': ObjectId('5b18b265d698289b41406ab9'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 945127826,
              'userId': 23437},
             {'_id': ObjectId('5b18b265d698289b41412364'),
           

              'movieId': 4,
              'rating': 4.0,
              'timestamp': 847362657,
              'userId': 43781},
             {'_id': ObjectId('5b18b282d698289b416d9797'),
              'movieId': 4,
              'rating': 5.0,
              'timestamp': 835892345,
              'userId': 43923},
             {'_id': ObjectId('5b18b282d698289b416e44af'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 1077748230,
              'userId': 44241},
             {'_id': ObjectId('5b18b282d698289b416e77ce'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 841496969,
              'userId': 44338},
             {'_id': ObjectId('5b18b283d698289b416fa0ac'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 901200910,
              'userId': 44876},
             {'_id': ObjectId('5b18b284d698289b417025c6'),
              'movieId': 4,
              'rating': 4.0,
            

              'movieId': 4,
              'rating': 5.0,
              'timestamp': 829224574,
              'userId': 71690},
             {'_id': ObjectId('5b18b2a9d698289b41aab034'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 843067368,
              'userId': 71912},
             {'_id': ObjectId('5b18b2a9d698289b41aaeed6'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 951112225,
              'userId': 72006},
             {'_id': ObjectId('5b18b2a9d698289b41ab4526'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 847478807,
              'userId': 72165},
             {'_id': ObjectId('5b18b2a9d698289b41ab97db'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 832783948,
              'userId': 72327},
             {'_id': ObjectId('5b18b2aad698289b41abc934'),
              'movieId': 4,
              'rating': 4.0,
             

              'rating': 4.0,
              'timestamp': 847976390,
              'userId': 85971},
             {'_id': ObjectId('5b18b2bed698289b41ca75e9'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 964550625,
              'userId': 86208},
             {'_id': ObjectId('5b18b2bed698289b41caebf7'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 911991611,
              'userId': 86429},
             {'_id': ObjectId('5b18b2bfd698289b41cc02ed'),
              'movieId': 4,
              'rating': 5.0,
              'timestamp': 864749089,
              'userId': 86890},
             {'_id': ObjectId('5b18b2c0d698289b41ccbe88'),
              'movieId': 4,
              'rating': 4.5,
              'timestamp': 1252445619,
              'userId': 87283},
             {'_id': ObjectId('5b18b2c0d698289b41cce035'),
              'movieId': 4,
              'rating': 5.0,
              'timestamp': 845606043,
  

             {'_id': ObjectId('5b18b2ead698289b410bc6c4'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 839759808,
              'userId': 115878},
             {'_id': ObjectId('5b18b2ead698289b410becd7'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 831431316,
              'userId': 115970},
             {'_id': ObjectId('5b18b2ead698289b410c054a'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 977078862,
              'userId': 116015},
             {'_id': ObjectId('5b18b2ead698289b410c5a31'),
              'movieId': 4,
              'rating': 5.0,
              'timestamp': 844753267,
              'userId': 116186},
             {'_id': ObjectId('5b18b2ead698289b410c7889'),
              'movieId': 4,
              'rating': 5.0,
              'timestamp': 851328295,
              'userId': 116223},
             {'_id': ObjectId('5b18b2ead698289b410c99d2'),
      

              'timestamp': 833389059,
              'userId': 129724},
             {'_id': ObjectId('5b18b2fed698289b412a4062'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 832415920,
              'userId': 129818},
             {'_id': ObjectId('5b18b2fed698289b412a804d'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 944920189,
              'userId': 129955},
             {'_id': ObjectId('5b18b2fed698289b412ab9f6'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 847870140,
              'userId': 130075},
             {'_id': ObjectId('5b18b2fed698289b412abbfa'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 949960231,
              'userId': 130082},
             {'_id': ObjectId('5b18b2fed698289b412abeb5'),
              'movieId': 4,
              'rating': 4.0,
              'timestamp': 940924856,
              'userId': 130

## <span style='background:yellow'>Your Turn</span>

In [8]:
db.tags.find_one()

{'_id': ObjectId('5b18b23cd698289b41045af7'),
 'movieId': 4141,
 'tag': 'Mark Waters',
 'timestamp': 1240597180,
 'userId': 18}

In [14]:
## Create a query that returns a single movie with it's tags 
## ---------------------------------------------------------


document = db.movies.aggregate([{
    # Let's join the ratings collection using movieId
    "$lookup": {
        "from": "tags",
        "localField": "movieId",
        "foreignField": "movieId",
        "as": "tags"
    }
},{
    "$limit": 1
}, {
    # Filter the ratings and specify our columns.
    # Note: There is no '*' specifier as in PostgresQL and most relational databases
    # you must explicitly list all desired columns if you project any of them.
    "$project": {
        "title": 1,
        "tags": 1
    }
}])


for document in document:
    pprint(document)


{'_id': ObjectId('5b18b242d698289b410b776c'),
 'tags': [{'_id': ObjectId('5b18b23dd698289b4104df4e'),
           'movieId': 4,
           'tag': 'chick flick',
           'timestamp': 1308558977,
           'userId': 9197},
          {'_id': ObjectId('5b18b23dd698289b4104df4f'),
           'movieId': 4,
           'tag': 'revenge',
           'timestamp': 1308558977,
           'userId': 9197},
          {'_id': ObjectId('5b18b23fd698289b4107540c'),
           'movieId': 4,
           'tag': 'characters',
           'timestamp': 1360563411,
           'userId': 58612},
          {'_id': ObjectId('5b18b23fd698289b4107540d'),
           'movieId': 4,
           'tag': 'chick flick',
           'timestamp': 1362769982,
           'userId': 58612},
          {'_id': ObjectId('5b18b23fd698289b41082505'),
           'movieId': 4,
           'tag': 'characters',
           'timestamp': 1335562009,
           'userId': 71833},
          {'_id': ObjectId('5b18b241d698289b410a6a76'),
           

In [16]:
## Create a query that returns the movie and it's tags, but only if it has a rating, limit your query to 5 results.
## ---------------------------------------------------------


documents = db.movies.aggregate([{
    # Let's join the ratings collection using movieId
    "$lookup": {
        "from": "ratings",
        "localField": "movieId",
        "foreignField": "movieId",
        "as": "ratings"
    }
}, {
    # Specify that we only want movies that have ratings
    "$match": {
        "ratings": {
            "$ne": []
        }
    }
}, {
    # Limit ourselves to one movie
    "$limit": 5
}, {
    # Bring in the links for our movie
    "$lookup": {
        "from": "tags",
        "localField": "movieId",
        "foreignField": "movieId",
        "as": "tags"
    }
}, {
    # Filter the ratings and specify our columns.
    # Note: There is no '*' specifier as in PostgresQL and most relational databases
    # you must explicitly list all desired columns if you project any of them.
    "$project": {
        "title": 1,
        "tags": 1
    }
}]);


for document in documents:
    pprint(document)

{'_id': ObjectId('5b18b242d698289b410b776c'),
 'tags': [{'_id': ObjectId('5b18b23dd698289b4104df4e'),
           'movieId': 4,
           'tag': 'chick flick',
           'timestamp': 1308558977,
           'userId': 9197},
          {'_id': ObjectId('5b18b23dd698289b4104df4f'),
           'movieId': 4,
           'tag': 'revenge',
           'timestamp': 1308558977,
           'userId': 9197},
          {'_id': ObjectId('5b18b23fd698289b4107540c'),
           'movieId': 4,
           'tag': 'characters',
           'timestamp': 1360563411,
           'userId': 58612},
          {'_id': ObjectId('5b18b23fd698289b4107540d'),
           'movieId': 4,
           'tag': 'chick flick',
           'timestamp': 1362769982,
           'userId': 58612},
          {'_id': ObjectId('5b18b23fd698289b41082505'),
           'movieId': 4,
           'tag': 'characters',
           'timestamp': 1335562009,
           'userId': 71833},
          {'_id': ObjectId('5b18b241d698289b410a6a76'),
           

           'userId': 58612},
          {'_id': ObjectId('5b18b23fd698289b41075435'),
           'movieId': 6,
           'tag': 'tough guy',
           'timestamp': 1280558828,
           'userId': 58612},
          {'_id': ObjectId('5b18b23fd698289b41077a2d'),
           'movieId': 6,
           'tag': 'realistic action',
           'timestamp': 1368285041,
           'userId': 58970},
          {'_id': ObjectId('5b18b23fd698289b410799f4'),
           'movieId': 6,
           'tag': '7.5-FilmAffinity',
           'timestamp': 1413194252,
           'userId': 63781},
          {'_id': ObjectId('5b18b23fd698289b410799f7'),
           'movieId': 6,
           'tag': 'es un poc de lo de sempre',
           'timestamp': 1422375285,
           'userId': 63781},
          {'_id': ObjectId('5b18b23fd698289b4107b8f2'),
           'movieId': 6,
           'tag': "Can't remember",
           'timestamp': 1153650935,
           'userId': 67145},
          {'_id': ObjectId('5b18b23fd698289b4107d23

          {'_id': ObjectId('5b18b241d698289b4109d997'),
           'movieId': 6,
           'tag': 'seen 2011',
           'timestamp': 1323123342,
           'userId': 110507},
          {'_id': ObjectId('5b18b241d698289b4109e1bd'),
           'movieId': 6,
           'tag': 'Robert De Niro',
           'timestamp': 1359325229,
           'userId': 111657},
          {'_id': ObjectId('5b18b241d698289b4109e1be'),
           'movieId': 6,
           'tag': 'visceral',
           'timestamp': 1359325240,
           'userId': 111657},
          {'_id': ObjectId('5b18b241d698289b4109e1c1'),
           'movieId': 6,
           'tag': 'Al Pacino',
           'timestamp': 1359325232,
           'userId': 111657},
          {'_id': ObjectId('5b18b241d698289b4109e1c4'),
           'movieId': 6,
           'tag': 'authenticity',
           'timestamp': 1359265989,
           'userId': 111657},
          {'_id': ObjectId('5b18b241d698289b4109e1c5'),
           'movieId': 6,
           'tag': 'gre

           'movieId': 10,
           'tag': 'spies',
           'timestamp': 1244207188,
           'userId': 81725},
          {'_id': ObjectId('5b18b240d698289b41088fd0'),
           'movieId': 10,
           'tag': 'secret service',
           'timestamp': 1244207184,
           'userId': 81725},
          {'_id': ObjectId('5b18b240d698289b41088fd5'),
           'movieId': 10,
           'tag': 'Pierce Brosnan',
           'timestamp': 1244207181,
           'userId': 81725},
          {'_id': ObjectId('5b18b240d698289b41088fd7'),
           'movieId': 10,
           'tag': 'espionage',
           'timestamp': 1244207169,
           'userId': 81725},
          {'_id': ObjectId('5b18b240d698289b41088fd8'),
           'movieId': 10,
           'tag': 'James Bond',
           'timestamp': 1244207167,
           'userId': 81725},
          {'_id': ObjectId('5b18b240d698289b4108bb0b'),
           'movieId': 10,
           'tag': 7,
           'timestamp': 1137204462,
           'userId':

In [17]:
# Be sure to run this cell when you are finished. Thank you.
client.close()

# Save your notebook, then `File > Close and Halt`

---